In [ ]:
import pdfplumber
import os
from PIL import Image
import shutil
import fitz
from sentence_transformers import SentenceTransformer, models, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from glob import glob
from llama_index.core.node_parser import SentenceSplitter
import torch
from torchvision import transforms
import chromadb
import uuid
from chromadb.config import Settings
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from IPython.display import display
import openai
import base64
from openai import OpenAI
from langchain import PromptTemplate
from llama_index.core.evaluation.multi_modal import MultiModalRelevancyEvaluator, MultiModalFaithfulnessEvaluator
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from IPython.display import Image, display
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModelForMaskedLM, AutoTokenizer
from fastembed import SparseTextEmbedding
import re
import json

from pdf2image import convert_from_path
from PIL import Image
import sys
import pyocr
import pyocr.builders

from sentence_transformers import SentenceTransformer, util
import nltk

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

import clip  # assuming you're using OpenAI's CLIP package
from PIL import Image
import torch



In [ ]:
pdf_paths = ['Dataset/File1.pdf', 'Dataset/File2.pdf', 'Dataset/File3.pdf', 'Dataset/File4.pdf']

output_folder = 'extracted_content/'
pdf_img_folder = 'pdf_images'
text_output_file = f'{output_folder}/pages.txt'
caption_save_path = f'{output_folder}/caption_pairs.json'

chunk_size = 50
chunk_overlap = 20

text_collection_name = 'CLIP_texts'
image_collection_name = 'CLIP_images'

# Clients Preparation

In [ ]:
os.environ["OPENAI_API_KEY"] = None

openai_client = OpenAI()

qdrant_client = QdrantClient(None)


# Texts & Images Extraction

In [ ]:
def clear_output_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f'Existing folder "{folder_path}" and its contents have been removed.')
    os.makedirs(folder_path, exist_ok=True)
    print(f'Folder "{folder_path}" is ready for new content.')

In [ ]:
def extract_text(txt_file, page, file_index, page_num, pdf_img_folder):
    text = page.get_text()
    lines = text.split('\n')
    non_empty_lines = [line for line in lines if line.strip()]
    cleaned_text = '\n'.join(non_empty_lines)
    txt_file.write(cleaned_text)
    txt_file.write("\n\n")

    single_page_file = f"{pdf_img_folder}/File{file_index}_Page{page_num+1}.txt"
    with open(single_page_file, "a", encoding="utf-8") as single_txt:
        single_txt.write(cleaned_text)

In [ ]:
def extract_images(doc, page_num, page, img_index, file_index, img_metas):
    image_list = page.get_images(full=True)

    if image_list:
        for _, img in enumerate(image_list, start=1):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"File{file_index}_Figure{img_index}.{image_ext}"
            image_path = os.path.join(output_folder, image_filename)
            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)
            img_index += 1
            img_metas.append([image_filename, page_num])
    
    else:
        print(f"No images found on page {page_num + 1}.")

    return img_index, img_metas
    

In [ ]:
def extract_images_and_text(pdf_paths, output_folder, text_output_file, pdf_img_folder):
    os.makedirs(output_folder, exist_ok=True)
    img_metas = []

    for file_index in range(len(pdf_paths)):
        pdf_path = pdf_paths[file_index]
        doc = fitz.open(pdf_path)

        with open(text_output_file, "a", encoding="utf-8") as txt_file:
            img_index = 1
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)

                extract_text(txt_file, page, file_index, page_num, pdf_img_folder)
                img_index, img_metas = extract_images(doc, page_num, page, img_index, file_index, img_metas)

    print(f"\nText extraction complete. Saved to '{text_output_file}'")
    print(f"Image extraction complete. Images saved in '{output_folder}'")

    return img_metas

In [ ]:
clear_output_folder(output_folder)
clear_output_folder(pdf_img_folder)
img_metas = extract_images_and_text(pdf_paths, output_folder, text_output_file, pdf_img_folder)

# Generate Images Captions

In [ ]:
for i in range(len(pdf_paths)):
    pdf_file = pdf_paths[i]
    pages = convert_from_path(pdf_file, dpi=200)
    for j, page in enumerate(pages):
        output_filename = f"File{i}_Page{j+1}.png"
        page.save(f"{pdf_img_folder}/{output_filename}", "PNG")

In [ ]:
img_pdf_pairs = []

for i in range(len(img_metas)):
    img_meta = img_metas[i]
    page = img_meta[1] + 1
    file = img_meta[0].split('_')[0]
    corresponding_pdf_page = f"{file}_Page{page}.png"
    corresponding_pdf_txt = f"{file}_Page{page}.txt"
    img_pdf_pairs.append([img_meta[0], [corresponding_pdf_page], [corresponding_pdf_txt]])

img_pdf_pairs[0][1].extend(
    sub[1][0] for sub in img_pdf_pairs[1:3]
)
img_pdf_pairs[0][2].extend(
    sub[2][0] for sub in img_pdf_pairs[1:3]
)
img_pdf_pairs[-1][1].extend(
    sub[1][0] for sub in img_pdf_pairs[-4:-2]
)
img_pdf_pairs[-1][2].extend(
    sub[2][0] for sub in img_pdf_pairs[-4:-2]
)

for i in range(1, len(img_pdf_pairs)-1):
    img_pdf_pairs[i][1].extend([
    img_pdf_pairs[i-1][1][0],
    img_pdf_pairs[i+1][1][0]
])
    img_pdf_pairs[i][2].extend([
    img_pdf_pairs[i-1][2][0],
    img_pdf_pairs[i+1][2][0]
])

In [ ]:
supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
caption_pairs = []

for i in range(len(img_pdf_pairs)):
    image_name = img_pdf_pairs[i][0]
    image_path = os.path.join(output_folder, image_name)
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    pdf_img_paths = []
    pdf_img_contexts = img_pdf_pairs[i][1]
    for j in range(len(pdf_img_contexts)):
        pdf_img = pdf_img_contexts[j]
        pdf_img_paths.append(os.path.join(pdf_img_folder, pdf_img))
    base64_pdf_imgs = []
    for pdf_img_path in pdf_img_paths:
        with open(pdf_img_path, "rb") as img_file:
            base64_pdf_img = base64.b64encode(img_file.read()).decode('utf-8')
            base64_pdf_imgs.append(base64_pdf_img)

    pdf_txt_paths = []
    pdf_txt_contexts = img_pdf_pairs[i][2]
    for j in range(len(pdf_txt_contexts)):
        pdf_txt = pdf_txt_contexts[j]
        pdf_txt_paths.append(os.path.join(pdf_img_folder, pdf_txt))
    txt_context = ""
    for pdf_txt_path in pdf_txt_paths:
        with open(pdf_txt_path, "r") as text_file:
            file_text = text_file.read()
            txt_context += file_text + "\n"

    messages = [
        {
            "role": "system",
            "content": (
                "You are an image reading expert. Summarize the description "
                "of the last image given the first three images as context."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "Below are three pdf images providing context, followed by a final image extracted from one of these pdfs."
                        "Please analyze the three context pdf images, then extract the symptom name and description of the image."
                        "The text content from the three pdf images are also provided below for your better understanding."
                        "Use the structure of Symptom: xxx, Description: xxx."
                    )
                },
                # First context image
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_pdf_imgs[0]}"
                    }
                },
                # Second context image
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_pdf_imgs[1]}"
                    }
                },
                # Third context image
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_pdf_imgs[2]}"
                    }
                },
                {
                    "type": "text",
                    "text": (
                        "Here are texts extracted from pdf images above: {txt_content}"
                    )
                },
                {
                    "type": "text",
                    "text": (
                        "Now here is the final image to summarize:"
                    )
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ]
        

    response = openai_client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=messages,
        max_tokens=300,
    )
    summary = response.choices[0].message.content
    caption_pairs.append((summary, image_path))
    print(f"----------{image_path}---------\n")
    print(summary)
    print('\n')


# Embed Texts

In [ ]:
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

pdf_text = read_text_file(text_output_file)

In [ ]:
def chunk_text(text, chunk_size=chunk_size, chunk_overlap=chunk_overlap):
    text_splitter = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    chunks = text_splitter.split_text(text)
    print(f"Total text chunks created: {len(chunks)}")
    return chunks

pdf_chunks = chunk_text(pdf_text)

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
image_embeddings = []
for caption_pair in caption_pairs:
    image = Image.open(caption_pair[1])

    image_input = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_embedding = model.encode_image(image_input)  

    image_embedding = image_embedding / image_embedding.norm(dim=-1, keepdim=True)
    image_embedding = image_embedding.squeeze(0)
    image_embeddings.append(image_embedding)

In [ ]:
image_paths = []
for i in range(len(caption_pairs)):
    image_paths.append(caption_pairs[i][1])

In [ ]:
text_dense_embeddings = []
with torch.no_grad():
    for chunk in pdf_chunks:
        text_input = clip.tokenize([chunk]).to(device)
        text_embedding = model.encode_text(text_input) 
        text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)
        text_embedding = text_embedding.squeeze(0)
        text_dense_embeddings.append(text_embedding)

In [ ]:
def get_text_sparse_embeddings(chunks):
    model = SparseTextEmbedding(model_name="prithivida/Splade_PP_en_v1")
    embeddings = list(model.embed(chunks))
    return embeddings

text_sparse_embeddings = get_text_sparse_embeddings(pdf_chunks)

# Save Text Embeddings to Vector Store

In [ ]:
if qdrant_client.collection_exists(collection_name=text_collection_name):
    qdrant_client.delete_collection(collection_name=text_collection_name)

qdrant_client.create_collection(
    collection_name=text_collection_name,
    vectors_config=models.VectorParams(
        size=len(text_dense_embeddings[1]),
        distance=models.Distance.COSINE,
    ),
    sparse_vectors_config={
        "sparse": models.SparseVectorParams()
    }
)

In [ ]:
def add_text_dense_embeddings(chunks, embeddings):
    for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
        doc_id = f"{uuid.uuid4()}" 
        qdrant_client.upsert(
            collection_name=text_collection_name,
            points=[
                models.PointStruct(
                    id=doc_id,
                    payload={
                        "type": "text-dense",
                        "chunk_text": chunk,
                        "chunk_index": i
                    },
                    vector=emb,
                ),
            ]
        )
add_text_dense_embeddings(pdf_chunks, text_dense_embeddings)

In [ ]:
def add_text_sparse_embeddings(chunks, sparse_embeddings):
    for i, (chunk, sparse_vector) in enumerate(zip(chunks, sparse_embeddings)):
        doc_id = f"{uuid.uuid4()}"
        qdrant_client.upsert(
            collection_name=text_collection_name,
            points=[
                models.PointStruct(
                    id=doc_id,
                    payload={
                        "type": "text-sparse",
                        "chunk_text": chunk,
                        "chunk_index": i
                    },
                    vector={
                        "sparse": {  
                            "indices": list(sparse_vector.indices),
                            "values": list(sparse_vector.values)
                        }
                    },
                ),
            ]
        )
add_text_sparse_embeddings(pdf_chunks, text_sparse_embeddings)

# Save Image Embeddings

In [ ]:
if qdrant_client.collection_exists(collection_name=image_collection_name):
    qdrant_client.delete_collection(collection_name=image_collection_name)

qdrant_client.create_collection(
    collection_name=image_collection_name,
    vectors_config=models.VectorParams(
        size=len(image_embeddings[1]),
        distance=models.Distance.COSINE,
    ),
    sparse_vectors_config={
        "sparse": models.SparseVectorParams()
    }
)

In [ ]:
def add_image_embeddings(embeddings, image_paths):
    for i, (emb, image_path) in enumerate(zip(embeddings, image_paths)):
        doc_id = f"{uuid.uuid4()}" 
        qdrant_client.upsert(
            collection_name=image_collection_name,
            points=[
                models.PointStruct(
                    id=doc_id,
                    payload={
                        "type": "images",
                        "image_index": i,
                        "image_path": image_path
                    },
                    vector=emb,
                ),
            ]
        )

In [ ]:
add_image_embeddings(image_embeddings, image_paths)